# Data Wrangling and Analyzing - Udacity Data Analysis Nanodegree

###### by Hager Mohamed

## Table of Content

- Problem Definition
- Dataset
- Data Gathering
- Data Assessment
    - visual assessment 
    - programatic assessment 
    - Quality issues
    - Tidiness issues
- Data cleanup
- visualization

### Problem Definition
Real-world data rarely comes clean. Using Python and its libraries, you will gather data from a variety of sources and in a variety of formats, assess its quality and tidiness, then clean it. This is called data wrangling. You will document your wrangling efforts in a Jupyter Notebook, plus showcase them through analyses and visualizations using Python (and its libraries) and/or SQL.

The dataset that you will be wrangling (and analyzing and visualizing) is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because "they're good dogs Brent." WeRateDogs has over 4 million followers and has received international media coverage.


### Dataset
#### Enhanced Twitter Archive

The WeRateDogs Twitter archive contains basic tweet data for all 5000+ of their tweets, but not everything. One column the archive does contain though: each tweet's text, which I used to extract rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo) to make this Twitter archive "enhanced." Of the 5000+ tweets, I have filtered for tweets with ratings only (there are 2356).

#### Image Predictions File

One more cool thing: I ran every image in the WeRateDogs Twitter archive through a neural network that can classify breeds of dogs*. The results: a table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images).

<b>So for the last row in that table:</b>

- tweet_id is the last part of the tweet URL after "status/" → https://twitter.com/dog_rates/status/889531135344209921
- p1 is the algorithm's #1 prediction for the image in the tweet → golden retriever
- p1_conf is how confident the algorithm is in its #1 prediction → 95%
- p1_dog is whether or not the #1 prediction is a breed of dog → TRUE
- p2 is the algorithm's second most likely prediction → Labrador retriever
- p2_conf is how confident the algorithm is in its #2 prediction → 1%
- p2_dog is whether or not the #2 prediction is a breed of dog → TRUE
etc.

#### Additional Data via the Twitter API

Back to the basic-ness of Twitter archives: retweet count and favorite count are two of the notable column omissions. Fortunately, this additional data can be gathered by anyone from Twitter's API or download the json file uploaded in the nanodegree resources. 

********************

### Data Gathering
1. Download archived file manually
2. Download image prediction file by code through request package
3. Download the twitter data programatically either through twitter API or through request from Udacity server 

In [ ]:
import pandas as pd
import numpy as np
import requests as req
import os
import json
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
'''
function downlowd file from given url

Input (str) represents url

Returns Downloaded file name
'''

def request_file(url):
    response = req.get(url)
    file_name = url.split('/')[-1]
    # save content of response in the file
    if not os.path.isfile(file_name):
        with open(file_name,'wb') as f:
            f.write(response.content)
    return file_name

In [ ]:
# read archived tweets

archive_df = pd.read_csv("twitter-archive-enhanced.csv")

In [ ]:
#download image prediction file

url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
file_name = request_file(url)

In [ ]:
#read image prediction file

prediction_df = pd.read_csv(file_name,sep='\t')

In [ ]:
json_url = "https://video.udacity-data.com/topher/2018/November/5be5fb7d_tweet-json/tweet-json.txt"
file_name = request_file(json_url)

In [ ]:
# import tweepy
# from tweepy import OAuthHandler
# from timeit import default_timer as timer

#  Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
#  These are hidden to comply with Twitter's API terms and conditions
# consumer_key = 'HIDDEN'
# consumer_secret = 'HIDDEN'
# access_token = 'HIDDEN'
# access_secret = 'HIDDEN'

# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)

# api = tweepy.API(auth, wait_on_rate_limit=True)

#  NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
#  df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
#  change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
#  NOTE TO REVIEWER: this student had mobile verification issues so the following
#  Twitter API code was sent to this student from a Udacity instructor
#  Tweet IDs for which to gather additional data via Twitter's API
# tweet_ids = df_1.tweet_id.values
# len(tweet_ids)

#  Query Twitter's API for JSON data for each tweet ID in the Twitter archive
# count = 0
# fails_dict = {}
# start = timer()
#  Save each tweet's returned JSON as a new line in a .txt file
# with open('tweet_json.txt', 'w') as outfile:
#     # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
#     for tweet_id in tweet_ids:
#         count += 1
#         print(str(count) + ": " + str(tweet_id))
#         try:
#             tweet = api.get_status(tweet_id, tweet_mode='extended')
#             print("Success")
#             json.dump(tweet._json, outfile)
#             outfile.write('\n')
#         except tweepy.TweepError as e:
#             print("Fail")
#             fails_dict[tweet_id] = e
#             pass
# end = timer()
# print(end - start)
# print(fails_dict)

In [ ]:
#create dataframe from json objects
json_list=[]

with open(file_name) as json_file:
    for obj in json_file:
        tweet = json.loads(obj)
        tweet_id = tweet['id']
        favorite_count = tweet['favorite_count']
        retweet_count = tweet['retweet_count']
        followers_count = tweet['user']['followers_count']
        json_list.append({'tweet_id':tweet_id, 
                          'favorite_count':favorite_count,
                          'retweet_count':retweet_count,
                          'followers_count':followers_count})

json_df = pd.DataFrame(json_list, columns= json_list[0].keys())

### Visual Assessment 
list the 3 dataframes for visual assessment 

In [ ]:
archive_df

In [ ]:
prediction_df

In [ ]:
# save json dataframe into csv format for visual assessment 
json_df.to_csv('twitter.csv',index=False)
json_df

### Programmatic Assessment 
using pandas functions to assess the data

In [ ]:
archive_df.info()

In [ ]:
archive_df.sample(5)

In [ ]:
archive_df.tweet_id.isnull().any()

In [ ]:
archive_df.tweet_id.duplicated().sum()

In [ ]:
archive_df.name.value_counts()

In [ ]:
# check rates validity
archive_df.rating_numerator.value_counts()

In [ ]:
archive_df.rating_numerator.describe()

In [ ]:
archive_df.query('rating_numerator == 204')['text']

In [ ]:
archive_df.query('rating_numerator ==204')['text'][1120]

In [ ]:
#check denominatior consistency
archive_df.rating_denominator.value_counts()


In [ ]:
prediction_df.info()

In [ ]:
prediction_df.sample(5)

In [ ]:
prediction_df[prediction_df.jpg_url.duplicated() & prediction_df.img_num.duplicated()]

In [ ]:
# rows that's not classified as dogs should be removed
prediction_df.query('p1_dog == False & p2_dog == False & p3_dog == False')

In [ ]:
json_df.info()

In [ ]:
json_df.describe()

In [ ]:
json_df[json_df.tweet_id.duplicated()]

#### Quality Issues
###### archive_df

1. Drop rows indicates to be retweet or reply
2. Drop data that doesn't have images in image prediction
3. useless coloumns need to be dropped
4. numerators and dominators datatypes need to be decimal
5. values of numerators and dominators need to be fixed as mentioned in text cause it impacts quality of rates
6. timestamp datatype should be converted into datetime object
7. dog names in name coloumn have many typos 
8. some rows have None value which makes it meaningliess and should be dropped
9. calculate rate from numerators and denominators
10. Replace none value with NAN value
###### prediction_df
11. some image urls and num are duplicated which means repeatition in data
12. some p#_dog are false as it's not even a dog

###### json_df
12. drop unused keys from json objects

#### tidiness_issues
1. archived_df 4 columns (dogger, floofer, pupper and puppo) for one variable (dog type)
2. predictions: the dog breed prediction could be packed into one column (breed_pred)
3. predictions: the prediction confidence could be packed into one column (pred_confidence)
4. spread timestamp into 3 columns day, month and year
5. merge all into one cleaned dataframe


## Data CleanUp


backup data into backup dataframes

In [ ]:
# databackup

archive_bkup = archive_df.copy()
prediction_bkup = prediction_df.copy()
tweetapi_bkup = json_df.copy()

#### Define
1. drop rows that indecates to be retweet or reply

#### Code

In [ ]:
archive_df = archive_df[archive_df.in_reply_to_status_id.isnull()]
archive_df = archive_df[archive_df.retweeted_status_id.isnull()]

#### Test

In [ ]:
archive_df.info()
archive_df.head()

******************

#### Define

2. fix datatype of timestamp coloumn to be datetime instead of string



#### Code

In [ ]:
archive_df['timestamp'] = pd.to_datetime(archive_df['timestamp'])

#### Test

In [ ]:
archive_df.info()

***********************
#### Define

3. split timestamp to 3 column

#### code

In [ ]:
archive_df['year'] = archive_df.timestamp.dt.year
archive_df['month'] = archive_df.timestamp.dt.month
archive_df['day'] = archive_df.timestamp.dt.day

#### Test


In [ ]:
archive_df.info()

****************************
#### Define
4. fix datatype of numerator and denominator to be decimal for accurate rating

#### code

In [ ]:
archive_df['rating_numerator'] = archive_df['rating_numerator'].astype(float)
archive_df['rating_denominator'] = archive_df['rating_denominator'].astype(float)

#### Test

In [ ]:
archive_df.info()

*************
#### Define
5. calculate rate

#### Code

In [ ]:
archive_df['rate'] = (archive_df.rating_numerator / archive_df.rating_denominator)

#### Test
archive_df.rate.value_counts()

***********
#### Define

6. clean unlogical rate

#### Code

In [ ]:
archive_df.query('rate == 2.7')

In [ ]:
archive_df.query('rate == 2.7')['text'][763]

In [ ]:
archive_df.loc[(archive_df.tweet_id == 778027034220126208), 'rate'] = 1.127
archive_df.rate.value_counts()

In [ ]:
archive_df.query('rate == 42')

In [ ]:
archive_df.query('rate ==42')['text'][2074]


In [ ]:
try:
    archive_df.drop(index=2074,inplace=True)
except:
    pass
archive_df.query('rate == 42')

In [ ]:
archive_df.query('rate ==0')

In [ ]:
archive_df.query('rate ==0')['text'][315]

In [ ]:
try:
    archive_df.drop(index=315,inplace=True)
except:
    pass
archive_df.query('rate == 0')


In [ ]:
archive_df.query('rate ==7.5')

In [ ]:
archive_df.query('rate ==7.5')['text'][695]

In [ ]:
archive_df.loc[(archive_df.tweet_id == 786709082849828864), 'rate'] = 0.975


In [ ]:
archive_df.query('rate == 2.6')

In [ ]:
archive_df.loc[(archive_df.tweet_id == 680494726643068929), 'rate'] = 1.126

In [ ]:
archive_df.query('rate == 177.600000')

In [ ]:
try:
    archive_df.drop(979,inplace=True)
except:
    pass
archive_df.query('rate == 177.600000')

In [ ]:
try:
    archive_df.drop(45,inplace=True)
except:
    pass
archive_df.query('tweet_id == 883482846933004288')

#### Test

In [ ]:
archive_df.rate.value_counts()

************
#### Define
8. concatenate dos type

#### Code

In [ ]:
archive_df.replace(to_replace='None', value='', inplace=True)
archive_df['dogs_type'] = archive_df['doggo'] + archive_df['pupper'] + archive_df['puppo'] + archive_df['floofer']



#### Test

In [ ]:
archive_df.query('tweet_id == 854010172552949760')


#### Define

9. Drop useless columns

#### Code

In [ ]:
try:
    archive_df.drop(['in_reply_to_status_id',
               'in_reply_to_user_id',
               'timestamp',
               'source',
               'retweeted_status_id',
               'retweeted_status_user_id',
               'retweeted_status_timestamp',
               'expanded_urls',
               'rating_numerator',
               'rating_denominator',
                    'doggo',
                    'floofer',
                    'pupper',
                    'puppo'],
               axis=1,inplace=True)
except:
    pass

#### Test

In [ ]:
archive_df

In [ ]:
prediction_df.info()

In [ ]:
duplicated_index = prediction_df[prediction_df.jpg_url.duplicated() & prediction_df.img_num.duplicated()].index
prediction_df.drop(duplicated_index,inplace=True)

In [ ]:
false_dogs = prediction_df.query('p1_dog == False & p2_dog == False & p3_dog == False').index
prediction_df.drop(false_dogs,inplace=True)

In [ ]:
archive_df = pd.merge(archive_df,prediction_df,how='inner', on="tweet_id")
archive_df.info()

In [ ]:
archive_df['p_dog_max'] = archive_df['p1_dog']
archive_df['p_max'] = archive_df['p1']
for i, row in archive_df.iterrows():
    archive_df['p_conf_max'] = archive_df[['p1_conf', 'p2_conf','p3_conf']].max(axis=1)
    if row['p1_conf'] == archive_df['p_conf_max'][i]:
        archive_df['p_dog_max'][i] = row['p1_dog']
        archive_df['p_max'][i] = row['p1']
    elif row['p2_conf'] == archive_df['p_conf_max'][i]:
        archive_df['p_dog_max'][i] = row['p2_dog']
        archive_df['p_max'][i] = row['p2']
    elif row['p3_conf'] == archive_df['p_conf_max'][i]:
        archive_df['p_dog_max'][i] = row['p3_dog']
        archive_df['p_max'][i] = row['p3']
    else:
        archive_df['p_dog_max'][i] = np.nan
        archive_df['p_max'][i] = np.nan
archive_df.info()

In [ ]:
try:
    archive_df.drop(['p1',
               'p1_conf',
               'p1_dog',
               'p2',
               'p2_conf',
               'p2_dog',
               'p3',
               'p3_conf',
               'p3_dog'
                    ],
               axis=1,inplace=True)
except:
    pass
archive_df

In [ ]:
json_df.info()

In [ ]:
archive_df = pd.merge(archive_df,json_df,how='inner', on="tweet_id")

In [ ]:
archive_df

## Store data

In [ ]:
archive_df.to_csv('twitter-cleaned.csv',index=False)


## Visualization

In [ ]:
plt.figure(figsize=(10,6))
archive_df['tweet_id'].groupby([archive_df['year'], archive_df['month']]).count().plot(kind='line')
plt.title('number of Tweets per month', size=20)
plt.xlabel('Time (Year, Month)')
plt.ylabel('Number of Tweets')

fig = plt.gcf()
fig.savefig('annual_tweets.png',bbox_inches='tight');


In [ ]:
plt.figure(figsize=(10,6))
archive_df['followers_count'].groupby([archive_df['year'],archive_df['month']]).count().plot(kind='line')
plt.title('number of followers per month', size=20)
plt.xlabel('Time (Year, Month)')
plt.ylabel('Number of followers')

fig = plt.gcf()
fig.savefig('annual_followers.png',bbox_inches='tight');


In [ ]:
plt.figure(figsize=(10,6))
archive_df.plot(x='retweet_count', y='favorite_count', kind='scatter')
plt.title('retweet & favourites correlation ', size=20)
plt.xlabel('retweet_count')
plt.ylabel('favorite_count')

fig = plt.gcf()
fig.savefig('retweet_favourite.png',bbox_inches='tight');

In [ ]:
plt.figure(figsize=(10,6))
archive_df.plot(x='retweet_count', y='followers_count', kind='scatter')
plt.xlabel('tweet id')
plt.ylabel('followers count')
plt.title('tweet_id and followers correlation')

fig = plt.gcf()
fig.savefig('retweet_follower.png',bbox_inches='tight');


In [ ]:
plt.figure(figsize=(10,6))
archive_df['rate'].value_counts().plot(kind='bar')
plt.title ('Rating Distribution', size=20)
plt.xlabel('Rate')
plt.ylabel('Number of Ratings')

fig = plt.gcf()
fig.savefig('rating_distribution.png',bbox_inches='tight');


In [ ]:
plt.figure(figsize=(10,6))
archive_df['dogs_type'].groupby(archive_df['rate']).value_counts().plot(kind='bar')
plt.title('rate of dog type', size=20)
plt.xlabel('dog type')
plt.ylabel('rate')

fig = plt.gcf()
fig.savefig('dog_type_rate.png',bbox_inches='tight');


In [ ]:
dog_type = archive_df.query('dogs_type == "doggo"')
plt.figure(figsize=(10,6))
dog_type['rate'].groupby([dog_type['year'],dog_type['month']]).count().plot(kind='line')
plt.title('rate of doggo type over the years', size=20)
plt.xlabel('Time (Year, Month)')
plt.ylabel('rate')

fig = plt.gcf()
fig.savefig('doggo_type_rate.png',bbox_inches='tight');


In [ ]:
dog_type = archive_df.query('dogs_type == "pupper"')
plt.figure(figsize=(10,6))
dog_type['rate'].groupby([dog_type['year'],dog_type['month']]).count().plot(kind='line')
plt.title('rate of pupper type over the years', size=20)
plt.xlabel('Time (Year, Month)')
plt.ylabel('rate')

fig = plt.gcf()
fig.savefig('pupper_type_rate.png',bbox_inches='tight');


In [ ]:
dog_type = archive_df.query('dogs_type == "floofer"')
plt.figure(figsize=(10,6))
dog_type['rate'].groupby([dog_type['year'],dog_type['month']]).count().plot(kind='line')
plt.title('rate of floofer type over the years', size=20)
plt.xlabel('Time (Year, Month)')
plt.ylabel('rate')

fig = plt.gcf()
fig.savefig('floofer_type_rate.png',bbox_inches='tight');


In [ ]:
dog_type = archive_df.query('dogs_type == "puppo"')
plt.figure(figsize=(10,6))
dog_type['rate'].groupby([dog_type['year'],dog_type['month']]).count().plot(kind='line')
plt.title('rate of puppo type over the years', size=20)
plt.xlabel('Time (Year, Month)')
plt.ylabel('rate')

fig = plt.gcf()
fig.savefig('puppo_type_rate.png',bbox_inches='tight');
